In [24]:
from tensorflow import keras

In [26]:
from tensorflow.keras.applications.resnet50 import ResNet50 

In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle



from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [3]:
from resnets_utils import *

In [5]:
'''
if there is a problem"ImportError: libGL.so.1: cannot open shared object file: No such file or directory"
solve it with:
apt-get update
apt-get install ffmpeg libsm6 libxext6  -y
'''
import cv2

In [12]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [ ]:
'''
error: OSError: Unable to open file (file signature not found)
sol:file is corrupted.
'''

In [27]:
img_height,img_width = 64,64 
num_classes = 6
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

In [28]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [31]:
from tensorflow.keras.optimizers import Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(learning_rate=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 64)

Epoch 1/100
17/17 [==============================] - 20s 705ms/step - loss: 3.4899 - accuracy: 0.1907
Epoch 2/100
17/17 [==============================] - 11s 668ms/step - loss: 3.0116 - accuracy: 0.2806
Epoch 3/100
17/17 [==============================] - 12s 693ms/step - loss: 2.7358 - accuracy: 0.3231
Epoch 4/100
17/17 [==============================] - 12s 729ms/step - loss: 2.3551 - accuracy: 0.4028
Epoch 5/100
17/17 [==============================] - 11s 658ms/step - loss: 2.1500 - accuracy: 0.4120
Epoch 6/100
17/17 [==============================] - 12s 704ms/step - loss: 1.7850 - accuracy: 0.5074
Epoch 7/100
17/17 [==============================] - 13s 782ms/step - loss: 1.4349 - accuracy: 0.5972
Epoch 8/100
17/17 [==============================] - 12s 676ms/step - loss: 1.2323 - accuracy: 0.6361
Epoch 9/100
17/17 [==============================] - 12s 685ms/step - loss: 0.9538 - accuracy: 0.7231
Epoch 10/100
17/17 [==============================] - 12s 686ms/step - loss: 0.730

In [33]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 2s 186ms/step - loss: 2.1698 - accuracy: 0.7167
Loss = 2.1698391437530518
Test Accuracy = 0.7166666388511658


In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

refrence:
https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33